# TFM Suministro Global - WGI de Pais - Neo4j - py2neo 
WGI- Workdwide Global Index es una ponderación de diferentes valores de índices de un país que da idea de la "seguridad política, judicial, económica y en general de la "fiabilidad de un Pais" a la hora de realizar transacciones comerciales con él  
**Cuanto más alto el valor WGI de ese país, más fiable e inseguro y por tanto necesitará gestionar esos riesgos**
Ejemplo: El mínimo de Finlandia ("Finland" **WGI= 1,98**) contrasta con el máximo de Corea del Norte ("Korea, Dem. Rep." **WGI=7,74**) con un promedio de 5,00 para los países reportados en este estudio de Suministro Global

## Partimos de la existencia, en paralelo a esta ejecución, de una instancia levantada y accesible de Neo4j  
### La primera vez que se quiera usar _py2neo_ erá  necesario asegurar su instalación:  _!pip install py2neo_

In [1]:
##!pip install py2neo 

In [1]:
#importar la librería para poder hacer uso de sus funcionalidades y objetos
from py2neo import *

In [2]:
#acceder a la instancia levantada de Neo4j - en nuestro caso instancia Docker - asignamos credenciales de autentificación -
graph_test01 = Graph(auth=('neo4j', 's3cr3t'))

In [3]:
#que es el objeto - graph_test01
graph_test01

Graph('bolt://localhost:7687')

In [4]:
#que es el objeto - tx
tx = graph_test01.begin()

In [5]:
tx

In [6]:
#query de conexión de prueba sin errores
# verificar existencia de NODOS de METERIALES Brutos y Refinados
#--- cuenta los nodos con diferentes tipos de etiqueta y los une en una única salida, 1 fila por cada tipo (name, cnt)
queryPRB= "MATCH (n)                                    \
RETURN labels (n)[0] as name, count (n) as Ntotal       \
UNION MATCH (n)                                         \
WITH labels (n)[1] as name, count (n) as Ntotal         \
WHERE name IS NOT NULL                                  \
RETURN name, Ntotal                                     \
"
graph_test01.run(queryPRB)

name,Ntotal
MaterialB,78
MaterialREF,78
Pais,116


## CARGAR DATOS

In [7]:
import numpy as np    #datos y operativa álgegra y matrices de n dimensiones (arrays)
import pandas as pd   #carga dataset y manipulación de datos  y dataframes

In [8]:
#DATOS: Fichero completo de MATERIAS PRIMAS CRÍTICAS en el suministro GLOBAL - Atendiendo al alcance del estudio - EU CRM 2020
#FICHERO: GLMateriales_cargar.csv
#CONTENIDO: Materiales 
#USO: Crear los NODOS de tipo "MaterialB" - Material bruto y "MaterialREF" - Material Refinado) 
## -- propiedades de cada material: nombre, abreviatura, familia y subfamilia 
## -- los datos de suministro -posteriores- fijarán las relaciones entre materiales, países proveedores y usos del material 
rutaFichero =  "https://raw.githubusercontent.com/JuanKFD/UNIR_TFM/main/GL-Paises-WGI_cargar.csv" 
dfPaises = pd.read_csv(rutaFichero, sep=';')


In [9]:
dfPaises.head()

,Pais,Pais_iso3,WGI_Pais
0,Algeria,DZA,"6,43"
1,Albania,ALB,"5,16"
2,Argentina,ARG,"5,47"
3,Armenia,ARM,"5,41"
4,Australia,AUS,"2,36"


In [10]:
dfPaises.tail()

,Pais,Pais_iso3,WGI_Pais
111,Uzbekistan,UZB,"6,98"
112,"Venezuela, Rb",VEN,"7,30"
113,Vietnam,VNM,"5,75"
114,Zambia,ZMB,"5,40"
115,Zimbabwe,ZWE,"7,17"


In [11]:
dfPaises.shape

(116, 3)

In [12]:
dfPaises.columns

Index(['Pais', 'Pais_iso3', 'WGI_Pais'], dtype='object')

In [13]:
nFilas = dfPaises.shape[0]

In [14]:
nFilas

116

### Cargados en memoria los _116 países con sus WGI para actualizar_ de la cadena de suministro global

# PREVISUALIZACION DE DATOS - PAISES
Aseguramos la accesibilidad y previsualización de los datos

In [15]:
for i in range(0, nFilas):
    print(dfPaises.Pais[i])
    print(dfPaises.Pais_iso3[i])
    print(dfPaises.WGI_Pais[i])
    


Algeria
DZA
6,43
Albania
ALB
5,16
Argentina
ARG
5,47
Armenia
ARM
5,41
Australia
AUS
2,36
Austria
AUT
2,50
Bahrain
BHR
5,15
Bangladesh
BGD
6,41
Belarus
BLR
6,18
Belgium
BEL
2,81
Bolivia
BOL
5,97
Brazil
BRA
5,08
Bulgaria
BGR
4,73
Burkina Faso
BFA
5,74
Burundi
BDI
7,00
Canada
CAN
2,26
Chile
CHL
3,11
China
CHN
5,83
Colombia
COL
5,39
Congo, D.R.
COD
7,60
Cote D_Ivoire
CIV
6,21
Cuba
CUB
5,87
Czech Republic
CZE
3,47
Denmark
DNK
2,11
Dominican Republic
DOM
5,40
Ecuador
ECU
5,99
Egypt, Arab Rep.
EGY
6,48
Estonia
EST
3,07
Ethiopia
ETH
6,52
Finland
FIN
1,98
France
FRA
3,11
Gabon
GAB
5,97
Georgia
GEO
4,47
Germany
DEU
2,47
Ghana
GHA
4,94
Greece
GRC
4,60
Guatemala
GTM
6,00
Guinea
GIN
6,72
Guyana
GUY
5,56
Hong Kong
HKG
2,56
Hungary
HUN
4,06
Iceland
ISL
2,52
India
IND
5,45
Indonesia
IDN
5,47
Iran, Islamic Rep.
IRN
6,65
Iraq
IRQ
7,35
Ireland
IRL
2,58
Israel
ISR
3,83
Italy
ITA
4,17
Jamaica
JAM
4,85
Japan
JPN
2,77
Jordan
JOR
5,16
Kazakhstan
KAZ
5,90
Kenya
KEN
6,03
Korea, Dem. Rep.
PRK
7,74
Korea, Rep.
KO

In [18]:
dfPaises.columns

Index(['Pais', 'Pais_iso3', 'WGI_Pais'], dtype='object')

### Seleccionar cada nodo de tipo :País para asignarle la NUEVA PROPIEDAD  WGI_Pais con su valor
**MATCH** (n:Pais)  
**WHERE** n.name = dfPaises.Pais[i]   
**SET** n.WGI_Pais = dfPaises.WGI_Pais[i]  
**RETURN** n   

Podríamos usar  otra opción, aunque en este caso evaluaría cada vez todos los nodos para **"casar cuando"** el nodo específico tenga el nombre de país que queremos y en ese caso establecería la propiedad específica, _creándola si no existiera_:         

**MATCH** (n:Pais)  
**SET** (**CASE WHEN** n.name = dfPaises.Pais[i] **THEN** n **END**).WGI_Pais = dfPaises.WGI_Pais[i]    
**RETURN** n.name, n.WGI_Pais

In [22]:
#  Crear /ACtualizar la PROPIEDAD WGI_Pais en el nodo PAIS por cada fila leída (paises con sus WGI)
## PREVISAULIZAR QUERYs antes de la ejecución
TipoNodo = " :Pais " 
for i in range(0, nFilas):
    strQry = "MATCH (p"                                               \
        + TipoNodo                                                     \
        + ")"                                                            \
        + " WHERE n.name = '"                                           \
        + dfPaises.Pais[i]                                               \
        + "'"                                                            \
        + " SET n.WGI_Pais = "                                            \
        + str(dfPaises.WGI_Pais[i]).replace(",",".")                      \
        + " RETURN p" 
    print("QRY [" + str(i) + "]> "+ strQry)
# querys  visualizadas

QRY [0]> MATCH (p :Pais ) WHERE n.name = 'Algeria' SET n.WGI_Pais = 6.43 RETURN p
QRY [1]> MATCH (p :Pais ) WHERE n.name = 'Albania' SET n.WGI_Pais = 5.16 RETURN p
QRY [2]> MATCH (p :Pais ) WHERE n.name = 'Argentina' SET n.WGI_Pais = 5.47 RETURN p
QRY [3]> MATCH (p :Pais ) WHERE n.name = 'Armenia' SET n.WGI_Pais = 5.41 RETURN p
QRY [4]> MATCH (p :Pais ) WHERE n.name = 'Australia' SET n.WGI_Pais = 2.36 RETURN p
QRY [5]> MATCH (p :Pais ) WHERE n.name = 'Austria' SET n.WGI_Pais = 2.50 RETURN p
QRY [6]> MATCH (p :Pais ) WHERE n.name = 'Bahrain' SET n.WGI_Pais = 5.15 RETURN p
QRY [7]> MATCH (p :Pais ) WHERE n.name = 'Bangladesh' SET n.WGI_Pais = 6.41 RETURN p
QRY [8]> MATCH (p :Pais ) WHERE n.name = 'Belarus' SET n.WGI_Pais = 6.18 RETURN p
QRY [9]> MATCH (p :Pais ) WHERE n.name = 'Belgium' SET n.WGI_Pais = 2.81 RETURN p
QRY [10]> MATCH (p :Pais ) WHERE n.name = 'Bolivia' SET n.WGI_Pais = 5.97 RETURN p
QRY [11]> MATCH (p :Pais ) WHERE n.name = 'Brazil' SET n.WGI_Pais = 5.08 RETURN p
QRY [12]

In [23]:
#comprobar que sigue abierta la conexión para enviar las queries a la BD 
graph_test01

Graph('bolt://localhost:7687')

In [25]:
# Crear un nodo por cada fila con sus propiedades
#nodos de PAIS,
# CREAR NODOS: "Pais" 
## query para crea nodo con sus  propiedades
TipoNodo = " :Pais " 
for i in range(0, nFilas):
    strQry = "MATCH (p"                                               \
        + TipoNodo                                                     \
        + ")"                                                            \
        + " WHERE p.name = '"                                           \
        + dfPaises.Pais[i]                                               \
        + "'"                                                            \
        + " SET p.WGI_Pais = "                                            \
        + str(dfPaises.WGI_Pais[i]).replace(",",".")                      \
        + " RETURN p" 
#ejecutar Query en BD abierta - Neo4j
    graph_test01.run(strQry)
#Visualizar Query ejecutada 
    print("EJECUTADA QRY [" + str(i) + "]> "+ strQry)
# nodos creados

EJECUTADA QRY [0]> MATCH (p :Pais ) WHERE p.name = 'Algeria' SET p.WGI_Pais = 6.43 RETURN p
EJECUTADA QRY [1]> MATCH (p :Pais ) WHERE p.name = 'Albania' SET p.WGI_Pais = 5.16 RETURN p
EJECUTADA QRY [2]> MATCH (p :Pais ) WHERE p.name = 'Argentina' SET p.WGI_Pais = 5.47 RETURN p
EJECUTADA QRY [3]> MATCH (p :Pais ) WHERE p.name = 'Armenia' SET p.WGI_Pais = 5.41 RETURN p
EJECUTADA QRY [4]> MATCH (p :Pais ) WHERE p.name = 'Australia' SET p.WGI_Pais = 2.36 RETURN p
EJECUTADA QRY [5]> MATCH (p :Pais ) WHERE p.name = 'Austria' SET p.WGI_Pais = 2.50 RETURN p
EJECUTADA QRY [6]> MATCH (p :Pais ) WHERE p.name = 'Bahrain' SET p.WGI_Pais = 5.15 RETURN p
EJECUTADA QRY [7]> MATCH (p :Pais ) WHERE p.name = 'Bangladesh' SET p.WGI_Pais = 6.41 RETURN p
EJECUTADA QRY [8]> MATCH (p :Pais ) WHERE p.name = 'Belarus' SET p.WGI_Pais = 6.18 RETURN p
EJECUTADA QRY [9]> MATCH (p :Pais ) WHERE p.name = 'Belgium' SET p.WGI_Pais = 2.81 RETURN p
EJECUTADA QRY [10]> MATCH (p :Pais ) WHERE p.name = 'Bolivia' SET p.WGI_P

EJECUTADA QRY [96]> MATCH (p :Pais ) WHERE p.name = 'Sudan' SET p.WGI_Pais = 7.70 RETURN p
EJECUTADA QRY [97]> MATCH (p :Pais ) WHERE p.name = 'Suriname' SET p.WGI_Pais = 5.16 RETURN p
EJECUTADA QRY [98]> MATCH (p :Pais ) WHERE p.name = 'Sweden' SET p.WGI_Pais = 2.05 RETURN p
EJECUTADA QRY [99]> MATCH (p :Pais ) WHERE p.name = 'Taiwan, China' SET p.WGI_Pais = 3.27 RETURN p
EJECUTADA QRY [100]> MATCH (p :Pais ) WHERE p.name = 'Tajikistan' SET p.WGI_Pais = 6.88 RETURN p
EJECUTADA QRY [101]> MATCH (p :Pais ) WHERE p.name = 'Tanzania' SET p.WGI_Pais = 5.74 RETURN p
EJECUTADA QRY [102]> MATCH (p :Pais ) WHERE p.name = 'Thailand' SET p.WGI_Pais = 5.50 RETURN p
EJECUTADA QRY [103]> MATCH (p :Pais ) WHERE p.name = 'Togo' SET p.WGI_Pais = 6.34 RETURN p
EJECUTADA QRY [104]> MATCH (p :Pais ) WHERE p.name = 'Tunisia' SET p.WGI_Pais = 5.40 RETURN p
EJECUTADA QRY [105]> MATCH (p :Pais ) WHERE p.name = 'Turkey' SET p.WGI_Pais = 5.34 RETURN p
EJECUTADA QRY [106]> MATCH (p :Pais ) WHERE p.name = 'Ugand

In [26]:
dfGraphMat= graph_test01.run("MATCH (n :Pais ) RETURN n").data()

In [27]:
dfGraphMat[0]


{'n': Node('Pais', Pais_iso3='DZA', WGI_Pais=6.43, name='Algeria')}

In [29]:
#validamos que todos los -nuevos- nodos se han creado :Pais  
graph_test01.run("MATCH (n :Pais) RETURN  LABELS(n), COUNT(n), COUNT(n.WGI_Pais)")

LABELS(n),COUNT(n),COUNT(n.WGI_Pais)
['Pais'],116,116


In [30]:
#validamos que todos los nodos se han creado (hasta este paso)
queryPRB= "MATCH (n)                                    \
RETURN labels (n)[0] as name, count (n) as Ntotal       \
UNION MATCH (n)                                         \
WITH labels (n)[1] as name, count (n) as Ntotal         \
WHERE name IS NOT NULL                                  \
RETURN name, Ntotal                                     \
UNION MATCH (n)                                         \
WITH labels (n)[2] as name, count (n) as Ntotal         \
WHERE name IS NOT NULL                                  \
RETURN  name, Ntotal                                    \
"
graph_test01.run(queryPRB)


name,Ntotal
MaterialB,78
MaterialREF,78
Pais,116


# fin proceso - Propiedad WGI_Pais creada en cada nodo  PAIS- CREADOS

In [81]:
# fin archivo